In [11]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from timeit import default_timer as timer
import time

# Plot
import matplotlib.pyplot as plt
import seaborn as sns

# Components of models
from sklearn.manifold import TSNE
from sklearn.svm import SVR
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    roc_curve,
    classification_report,
    precision_recall_curve,
)
from sklearn.model_selection import (
    cross_val_score,
    GridSearchCV,
    train_test_split,
    KFold,
)
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.utils import resample, shuffle

# Evaluate result of models
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve

# Dealing with classification with imbalance classes
from imblearn.pipeline import Pipeline, make_pipeline
from imblearn.over_sampling import SMOTE

# Storing model
from joblib import Parallel, delayed
import joblib

# pickled_model = joblib.load(open('model.joblib', 'rb'))

# df = pd.DataFrame(
#     {
#         "credit_policy": [1, 1],
#         "purpose": ["debt_consolidation", "credit_card"],
#         "int_rate": [0.1189, 0.1071],
#         "installment": [829.1, 228.22],
#         "log_annual_inc": [11.35040654, 11.08214255],
#         "dti": [19.48, 14.29],
#         "fico": [737, 707],
#         "days_with_cr_line": [5639.958333, 2760],
#         "revol_bal": [28854, 33623],
#         "revol_util": [52.1, 76.7],
#         "inq_last_6mths": [0, 0],
#         "delinq_2yrs": [0, 0],
#         "pub_rec": [0, 0],
#     }
# )

# Load data from the CSV file
df = pd.read_csv('history_loan_data.csv', index_col=None)

# Change the dots in the column names to underscores
df.columns = [col.replace(".", "_") for col in df.columns]

# Split columns by data type
num = [
    "credit_policy",
    "int_rate",
    "installment",
    "log_annual_inc",
    "dti",
    "fico",
    "days_with_cr_line",
    "revol_bal",
    "revol_util",
    "inq_last_6mths",
    "delinq_2yrs",
    "pub_rec",
]
non_num = ["purpose"]


# Log transform function
def log_transform(data, to_log):
    X = data.copy()
    for item in to_log:
        # Add 1 to the data to prevent infinity values
        X[item] = np.log(1 + X[item])
    return X


df["purpose"].unique()

df_e = pd.get_dummies(data=df)
# df_e = df.drop(["purpose_all_other"], axis=1)

# Log transform
to_log = [
    "credit_policy",
    "int_rate",
    "installment",
    "dti",
    "fico",
    "days_with_cr_line",
    "revol_bal",
    "revol_util",
    "inq_last_6mths",
    "delinq_2yrs",
    "pub_rec",
]
df_e_l = log_transform(df_e, to_log)

df_e_l.head()

scaler = StandardScaler()
scaler.fit(df_e_l)
tmp_train = scaler.transform(df_e_l)
df_e_l_n = pd.DataFrame(data=tmp_train, index=df_e.index, columns=df_e.columns)
df_e_l_n.head()





# result = pickled_model.predict(df)
# print(result)


c:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_

,credit_policy,int_rate,installment,log_annual_inc,dti,fico,days_with_cr_line,revol_bal,revol_util,inq_last_6mths,delinq_2yrs,pub_rec,not_fully_paid,purpose_all_other,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_major_purchase,purpose_small_business
0,0.492222,-0.127784,1.624552,0.680388,0.835518,0.709712,0.615406,0.762598,0.426346,-1.057798,-0.344675,-0.245681,-0.436524,-0.567142,-0.389558,1.191856,-0.192721,-0.265117,-0.218647,-0.262855
1,0.492222,-0.571733,-0.138685,0.244031,0.430905,-0.076003,-0.567258,0.832981,0.771897,-1.057798,-0.344675,-0.245681,-0.436524,-0.567142,2.567010,-0.839028,-0.192721,-0.265117,-0.218647,-0.262855
2,0.492222,0.496266,0.509438,-0.908659,0.166298,-0.756630,0.317193,-0.206522,-0.201133,-0.014729,-0.344675,-0.245681,-0.436524,-0.567142,-0.389558,1.191856,-0.192721,-0.265117,-0.218647,-0.262855
3,0.492222,-0.810698,-0.602969,0.680388,-0.287535,0.057235,-0.603652,0.833583,0.730059,-0.014729,-0.344675,-0.245681,-0.436524,-0.567142,-0.389558,1.191856,-0.192721,-0.265117,-0.218647,-0.262855
4,0.492222,0.749902,-1.222582,0.597961,0.491147,-1.177070,0.073870,-0.068449,0.180466,-1.057798,2.109607,-0.245681,-0.436524,-0.567142,2.567010,-0.839028,-0.192721,-0.265117,-0.218647,-0.262855
